In [1]:
import numpy as np
import pandas as pd



In [2]:
contract = .0185
#MAKE THIS THE PREVIOUS DAY YOU NEED
val = '2022-05-17'


In [3]:
#prev = pd.read_csv('previous.csv')

In [4]:
pp = pd.read_csv('export.csv', skiprows = 1,header = None)   
df = pd.read_csv('invoice.csv')


In [5]:
def color_negative_red(value):
    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [6]:
invoice = df.filter(['Invoice Number','Ship Date','Terminal Name','SCAC','Product Description','Billed Quantity','Price'], axis=1)



In [7]:
invoice['Date'] = invoice['Ship Date'].str[:10]
invoice['Ship Date'] = invoice['Ship Date'].str[11:]
invoice['Ship Date'] = invoice['Ship Date'].str[:5]
invoice['Time'] = invoice["Ship Date"].str.replace(":","")
invoice['Time'] = invoice['Time'].astype(int)
invoice['Time'] = invoice['Time'] - 400
invoice = invoice.drop(columns=['Ship Date'])


In [8]:
#invoice = invoice.groupby(['Invoice Number', 'Terminal Name', 'SCAC', 'Product Description','Billed Quantity','Price','Date','Time'])

In [9]:
#invoice

In [10]:
export = pp.filter([2,6,12,13,14,16,17,22 ], axis=1)
export = export.rename({2: 'Company', 6: 'Terminal Name', 12: 'Product Description',13: 'Date',14: 'Time', 16:'Price', 17:'d1',22:'tax'}, axis=1)

In [11]:
export['Date']= export['Date'].map(str)

In [12]:
export['Year'] = export['Date'].str[:4]
export['Date'] = export['Date'].str[4:]
export['Day'] = export['Date'].str[2:]
export['Month'] = export['Date'].str[:2]
export['Date'] = export['Year'].map(str) + '-' + export['Month'].map(str) + '-' + export['Day'].map(str)



In [13]:

marathon = export.loc[export['Company'] == 'Marathon Unbranded']
marathon = marathon.drop(columns=['Company'])

In [14]:
marathon.loc[marathon['Terminal Name'] == 'Marathon Petroleum Corporation', 'Terminal Name'] = 'Vancouver, WA - MPC Terminal'
marathon.loc[marathon['Terminal Name'] == 'Kinder Morgan Energy Partners South', 'Terminal Name'] = 'Portland, OR - Kinder Morgan Willbridge Terminal'

In [15]:
#MPC
marathon.loc[marathon['Product Description'] == 'ULS #2', 'Product Description'] = 'ULSD No 2'
marathon.loc[marathon['Product Description'] == 'ULS 2 DYED', 'Product Description'] = 'ULSD No 2 Dyed'
marathon.loc[marathon['Product Description'] == 'ULS#2 DYE B5', 'Product Description'] = 'ULSD No 2 Bio 5% Dyed'
marathon.loc[marathon['Product Description'] == 'Conv 87 Unleaded with 10% Ethanol', 'Product Description'] = '87 Reg GHL 10% Eth 7.8'
#Kinder
marathon.loc[marathon['Product Description'] == 'ULS #2 B5', 'Product Description'] = 'ULSD No 2 Bio 5%'
marathon.loc[marathon['Product Description'] == 'Conv 87 Unleaded Octane', 'Product Description'] = '87 Regular 7.8'



In [16]:
prev = marathon

In [17]:

prev = prev.loc[prev['Date'] == val]
laster = prev
marathon = marathon.loc[marathon['Date'] == invoice.values[0][6]]
prev =prev.drop_duplicates(subset=['Terminal Name', 'Product Description'], keep='last')

prev['Time'] = '0001'
prev['Date'] =  invoice.values[0][6]


C:\Users\andrewl\Continuum\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\andrewl\Continuum\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
laster =laster.drop_duplicates(subset=['Terminal Name', 'Product Description'], keep='last')



In [19]:
marathon = marathon.append(prev)
marathon['Time'] = marathon['Time'].astype(int)

In [20]:
marathon = marathon.sort_values(by="Time",ascending=False)

In [21]:
final = pd.merge(invoice, marathon, how="left", on=["Terminal Name", "Product Description"])

In [22]:
final['Time_x'] = final['Time_x'].astype(float)

In [23]:
final = final[final['Time_x']>final['Time_y']]

In [24]:
final = final.sort_values(by=['Time_x','Time_y'],ascending = [False, True])

In [25]:
final =final.drop_duplicates(subset=['Terminal Name', 'Product Description','Time_x','Billed Quantity'], keep='last')

In [26]:
final['Discount2'] = contract

In [27]:
final['d1'] = final['d1'].astype(float)
final['Price_y'] = final['Price_y'].astype(float)
final['Discount2'] = final['Discount2'].astype(float)
final['tax'] = final['tax'].astype(float)

In [28]:
final['Expected Price'] = final['Price_y'] - final['tax'] - final['Discount2']

In [29]:
final['Expected Price'] = final['Expected Price'].astype(float)
final['Price_x'] = final['Price_x'].astype(float)

In [30]:
final['Price Difference'] = final['Expected Price'] - final['Price_x']

In [31]:
final =final.round({'Price Difference': 6})

In [32]:
final.rename(columns = {'Price_x':'Billed Price', 'Price_y':'Marathon Price'}, inplace = True)
final.rename(columns = {'Price_x':'Billed Price', 'tax':'Contract'}, inplace = True)
final.rename(columns = {'Time_x':'Time Fueled', 'Time_y':'Marathon Time','d1':'Tax'}, inplace = True)
final.rename(columns = {'Date_x':'Date Fueled', 'Date_y':'Marathon Date'}, inplace = True)
final = final[final['SCAC'] != 'JUDB']



In [33]:

laster.filter(items=["Terminal Name","Marathon Time","Marathon Price"]).head()

,Terminal Name
288,"Vancouver, WA - MPC Terminal"
289,"Vancouver, WA - MPC Terminal"
290,"Vancouver, WA - MPC Terminal"
291,"Vancouver, WA - MPC Terminal"
292,"Vancouver, WA - MPC Terminal"


In [34]:
final = final[['Invoice Number','Terminal Name','Product Description','Time Fueled','Marathon Time','Billed Price','Contract','Discount2','Marathon Price','Expected Price','Price Difference']]

In [35]:
finder = final

In [36]:
final = final.style.applymap(color_negative_red, subset=['Expected Price','Billed Price','Price Difference'])

In [37]:
final.to_excel('final.xlsx', index=False)